# PRIMAP-hist data Preparation

This jupyter notebook sets out a method for preparing PRIMAPhist data for plotting. It can further be used as a guide and basis for setting up other input datasets. 

The dataset can be viewed and accessed at:
https://www.pik-potsdam.de/paris-reality-check/primap-hist/

Gütschow, J.; Jeffery, L.; Gieseke, R. (2019): The PRIMAP-hist national historical emissions time series (1850-2016). v2.0. GFZ Data Services. https://doi.org/10.5880/pik.2019.001

The downloaded data should be stored in the 'input-data' folder of this repository. PRIMAP-hist is updated roughly once per year. The user should therefore check for updates at the above link and download new data when available. If this is done, the new filenames will need to be added below. 

------
Possible improvments:
*  unit conversion
* warn of over-writing files AND/OR automise file names.  Currently it's possible to give a file a poor name. Would require a look-up for category names though...


In [1]:
# import modules

# system
import re
import sys
import os

# data handling
import pandas as pd
import numpy as np

# open climate data packages
from countrygroups import UNFCCC, EUROPEAN_UNION, ANNEX_ONE, NON_ANNEX_ONE
from shortcountrynames import to_name

# global stocktake tools
import gst_tools.gst_utils as utils

In [2]:
## User definition of requirements

include_extrapolated_data = True

# to understand the codes to use here, please see the PRIMAP-hist documentation
raw_entity = 'CO2'
raw_sector = 'IPCM0EL'
raw_scenario = 'HISTCR' 

# choose something useful! These will be used to generate the new filename.
new_variable_name = 'CO2-total-excl-LU'
new_source_name = 'PRIMAP-hist_v2.0'

# Based on countrygroups package, select the group of countries you would like to extract. 
# Note that the raw data also includes groups.
needed_countries = UNFCCC

# First year of data needed for further plotting
start_year = 1990 


In [3]:
## reduce the data

# select the right file 
# !*** Note that the file names here will need to be updated for new versions of PRIMAP-hist ***! 
if include_extrapolated_data:
    raw_data_file = 'PRIMAP-hist_v2.0_11-Dec-2018.csv'
else:
    raw_data_file = 'PRIMAP-hist_v2.0_no_extrapolation_11-Dec-2018.csv'
    

# get the data
raw_data_folder = 'input-data'
fname = os.path.join('', raw_data_folder, raw_data_file)
print('reading ' + fname)
raw_data = pd.read_csv(fname)

# reduce to only the desired variable (one per output file)
new_data = raw_data.loc[(raw_data['entity'] == raw_entity) & 
                        (raw_data['scenario'] == raw_scenario) &
                        (raw_data['category'] == raw_sector)
                       ]

# reduce the countries or regions to only those desired
new_data = new_data.loc[raw_data['country'].isin(needed_countries)]

# tell the user if any of the needed countries are missing and, if yes, which ones:
missing_countries = list(set(needed_countries) - set(new_data['country'].unique()))
if missing_countries:
    print('Not all countries requested were available in the raw data. You are missing the following:')
    for country in missing_countries:
        print('   ' + to_name(country))
    print('---------')


# reduce to only required years
new_data = utils.change_first_year(new_data, start_year)

# rename columns to follow conventions
new_data = new_data.rename(columns={'entity': 'variable'})

# make sure 'variable' contains all necessary information
new_data['variable'] = new_variable_name

# label the source
new_data['source'] = new_source_name

new_data = utils.check_column_order(new_data)

# check! 
new_data

reading input-data/PRIMAP-hist_v2.0_11-Dec-2018.csv
Not all countries requested were available in the raw data. You are missing the following:
   Palestine
   European Union
---------
First year of data available is now 1990
Last year of data available is 2016


,category,country,scenario,source,unit,variable,1990,1991,1992,1993,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,IPCM0EL,AFG,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,2850.00,2640.00,1600.00,1530.00,...,2430.0,4360.0,6930.0,8640.00,12400.0,11000.0,10200.0,11300.0,12100.0,14200.0
1,IPCM0EL,AGO,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,5130.00,5110.00,5210.00,5800.00,...,25100.0,25600.0,27600.0,28900.00,30200.0,33300.0,32500.0,34600.0,37900.0,39600.0
2,IPCM0EL,ALB,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,3950.00,5570.00,3360.00,3270.00,...,4520.0,4600.0,5010.0,5260.00,5960.0,5560.0,5700.0,6440.0,5750.0,5990.0
3,IPCM0EL,AND,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,430.00,437.00,446.00,455.00,...,599.0,597.0,575.0,579.00,551.0,545.0,536.0,521.0,539.0,534.0
4,IPCM0EL,ARE,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,46400.00,50800.00,51800.00,58500.00,...,164000.0,190000.0,202000.0,195000.00,201000.0,214000.0,206000.0,255000.0,275000.0,282000.0
5,IPCM0EL,ARG,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,111000.00,119000.00,123000.00,123000.00,...,184000.0,188000.0,182000.0,190000.00,199000.0,198000.0,203000.0,205000.0,207000.0,207000.0
6,IPCM0EL,ARM,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,21800.00,20400.00,25000.00,9970.00,...,5140.0,5760.0,4680.0,4670.00,5450.0,6300.0,5970.0,6000.0,6810.0,6930.0
7,IPCM0EL,ATG,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,331.00,316.00,314.00,322.00,...,533.0,545.0,575.0,590.00,580.0,592.0,593.0,607.0,626.0,636.0
8,IPCM0EL,AUS,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,279000.00,281000.00,286000.00,290000.00,...,402000.0,406000.0,410000.0,408000.00,405000.0,408000.0,399000.0,395000.0,404000.0,415000.0
9,IPCM0EL,AUT,HISTCR,PRIMAP-hist_v2.0,Gg,CO2-total-excl-LU,62800.00,66300.00,60800.00,61100.00,...,74300.0,74000.0,67700.0,72600.00,70400.0,68000.0,68200.0,64400.0,66900.0,67600.0


In [4]:
## write the data to file

"""
First ensure that years, unit, 'country', and variable are all in data. If they are
can proceed to print data
"""

if 'country' not in new_data.columns or 'unit' not in new_data.columns:
    
    print('Missing required information! Please check your input data and processing!')
    
else:
    
    # define filename as composite of variable and source name
    fname_out = new_source_name + '_' + new_variable_name + '.csv' 
    fullfname_out = os.path.join('proc-data', fname_out)

    # check folder exists
    if not os.path.exists('proc-data'):
        os.makedirs('proc-data')

    # write to csv in proc data folder
    new_data.to_csv(fullfname_out, index=False)

    # celebrate success 
    print('Processed data written to file! - ' + fullfname_out)
    

Processed data written to file! - proc-data/PRIMAP-hist_v2.0_CO2-total-excl-LU.csv


**Test ground below**

In [6]:
new_data.columns


Index(['category', 'country', 'scenario', 'source', 'unit', 'variable', '1990',
       '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')